#### DRW - Crypto Market Prediction

Develop a model capable of predicting crypto future price movements.

Link to the competition: https://www.kaggle.com/competitions/drw-crypto-market-prediction/overview

The evaluation metric is: Pearson correlation coefficient (PCC)

In [1]:
import numpy as np
import polars as pls
from pathlib import Path
import lightgbm as lgb

import plotly.express as px

In [2]:
data_path = "/home/yang/team_kaggle/drw/data"

In [3]:
training_data = pls.read_parquet(Path(data_path, "train.parquet"))
# Shuffle the training set
training_data = training_data.sample(fraction=1.0, shuffle=True, seed=42)
training_data.head(10)

bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,…,X856,X857,X858,X859,X860,X861,X862,X863,X864,X865,X866,X867,X868,X869,X870,X871,X872,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label,timestamp
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns]
10.201,3.554,137.504,477.139,614.643,0.455945,0.156054,0.22047,0.314345,0.213229,0.309831,0.342415,0.364929,0.782787,0.437171,0.558691,0.719275,0.934595,0.594639,0.624011,0.634845,1.572796,0.356922,0.610073,0.471605,0.617061,0.221393,0.160492,0.188,1.6113,0.082352,0.13999,0.225973,0.172749,-0.11272,-0.170855,-0.065223,…,1.221426,-0.186892,0.493867,-2.2141e-34,0.44962,-0.164752,0.183902,0.102021,0.0,1.592214,-0.445619,0.0,-0.086006,0.0,0.0,0.0,0.0,0.523175,0.22928,2.799699,4.053666,0.22096,0.22589,0.473638,0.521432,0.583462,1.262546,1.557144,1.471337,1.471337,1.471337,1.471337,1.468739,1.410402,2.360201,0.513817,2023-06-25 11:04:00
7.955,12.834,9.427,11.389,20.816,-0.296205,-0.077624,-0.164131,-0.217358,-0.122609,-0.111764,-0.147973,-0.194297,-0.014491,0.088987,-0.004808,0.029109,0.222959,0.209831,0.137743,0.061467,0.804471,0.133721,0.2913,0.223908,0.286692,-0.07402,-0.256348,-0.260209,1.204841,-0.052857,0.042527,0.091091,0.062379,-0.099559,-0.092654,0.013202,…,0.140121,0.10829,0.416409,-0.034783,-0.206572,0.717057,-1.53189,0.394755,0.0,1.067378,-0.441767,0.0,0.044831,0.0,0.0,0.0,0.0,0.627796,0.248407,2.636226,3.711776,0.176848,0.215451,0.533364,0.752753,1.123312,2.353131,2.262618,0.000688,0.000681,0.010116,0.060824,0.113785,0.099791,0.208067,0.639713,2023-06-16 02:41:00
16.253,8.612,316.883,69.078,385.961,-0.072822,-0.30619,0.144202,0.186061,0.052609,-0.051571,-0.082299,-0.089195,0.120548,-0.198927,0.002799,0.00918,-0.063864,-0.094576,-0.080923,-0.048403,1.11094,0.017592,0.550179,0.489121,0.66136,0.391317,0.303432,0.136173,0.952494,0.149705,0.678679,1.098371,1.323821,0.623288,0.425273,0.082367,…,0.182098,0.142667,0.368144,0.363662,0.273017,-0.554033,-0.824959,0.167162,0.0,1.160456,-0.061886,0.0,0.133485,0.0,0.0,0.0,0.0,0.679556,0.24292,2.566681,4.341193,0.446872,0.499126,1.074451,1.121207,1.136961,2.211898,2.084438,0.000714,0.000612,0.000432,0.00029,0.005688,0.02585,0.212906,-0.171524,2023-03-01 22:00:00
3.778,4.864,213.121,170.398,383.519,0.043671,-0.249998,-0.069011,0.177828,0.158949,0.163474,0.14469,0.144913,0.436186,-0.183944,0.281176,0.730603,1.035081,0.584057,0.586901,0.604066,0.059724,0.446625,1.459665,1.055542,1.283784,0.678095,0.541884,0.441548,0.559886,0.918481,0.894511,1.194638,1.420389,0.640372,0.257607,-0.003234,…,-0.212133,0.203845,0.569276,-1.407489,0.659926,-0.44841,0.340363,0.000176,0.0,0.192575,0.70822,0.0,-0.363277,0.0,0.0,0.0,0.0,0.570799,0.203565,2.924183,4.176899,0.772395,0.579167,0.863318,0.633419,0.591128,1.488359,1.745534,0.000544,0.000468,0.000496,0.003245,0.040697,0.05966,0.160684,0.031799,2023-03-24 08:56:00
5.185,5.32,72.752,15.147,87.899,0.366187,-0.846847,-0.951879,-0.682562,-0.082663,0.262573,0.254918,0.231903,0.925197,-0.439139,-0.365102,0.076913,0.70938,0.664166,0.641591,0.618421,1.385803,-0.076158,0.087634,0.170724,0.373188,0.365099,0.433553,0.507511,1.239519,-0.381761,-0.089616,0.07396,0.34021,0.452311,0.585439,0.329714,…,-0.142669,0.23923,0.397981,-0.260967,0.374333,-0.15952,0.072865,-0.618188,0.0,1.426134,0.566085,0.0,0.219349,0.0,0.0,0.0,0.0,0.429022,0.24495,2.139902,4.216017,0.385669,0.432073,0.809794,0.791336,0.823082,1.508344,1.418105,0.000448,0.000764,0.021119,0.16817,0.683066,0.960149,2.085619,-0.038752,2023-11-27 03:14:00
3.471,10.965,118.344,234.565,352.909,-0.01295,-0.916387,-1.001151,-0.979312,-0.452772,-0.310239,-0.184183,-0.082118,

In [4]:
print("Size of training data (GB):", training_data.estimated_size("gb"))

Size of training data (GB): 3.5145926401019096


In [5]:
# Count null values in each column
null_counts = training_data.select([pls.col(column).null_count().alias(column) for column in training_data.columns])
null_counts

bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,…,X856,X857,X858,X859,X860,X861,X862,X863,X864,X865,X866,X867,X868,X869,X870,X871,X872,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label,timestamp
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
training_split = int(len(training_data) * 0.8)
validation_split = int(len(training_data) * 0.9)

In [7]:
# Set parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [ ]:
#### Train a LightGBM model
evals_result = {}

train_feature_list = training_data.columns[:-2]  # Exclude 'label' and 'timestamp'

training_data_loader = lgb.Dataset(training_data.select(pls.col(train_feature_list))[:training_split],
                                   label=training_data.select(pls.col("label"))[:training_split].to_numpy())

validation_data_loader = lgb.Dataset(training_data.select(pls.col(train_feature_list))[training_split:validation_split],
                                     label=training_data.select(pls.col("label"))[training_split:validation_split].to_numpy())

model = lgb.train(params, training_data_loader, num_boost_round=10,
                  valid_sets=[training_data_loader, validation_data_loader],
                  valid_names=['train', 'val'],
                  callbacks=[lgb.early_stopping(stopping_rounds=5), lgb.record_evaluation(evals_result)],
    )

training_loss = evals_result['train']['rmse'][-1]
validation_loss = evals_result['val']['rmse'][-1]
print("Training Losses per iteration:", training_loss)
print("Validation Losses per iteration:", validation_loss)

model.save_model('drw_lgbm.txt')

In [11]:
importances = model.feature_importance(importance_type='gain')
feature_importance_df = pls.DataFrame({
    "feature": train_feature_list,
    "importance": importances
}).sort("importance", descending=True)

feature_importance_df.head(20)

px.bar(feature_importance_df.head(20).to_pandas(), x="feature", y="importance", title="Top 20 Feature Importances (Gain)").show()

#### Model evaluation

In [ ]:
# load saved model to make predictions
model = lgb.Booster(model_file='drw_lgbm.txt')

y_pred = model.predict(training_data.select(pls.col(train_feature_list))[validation_split:])
y_pred

In [ ]:
import matplotlib.pyplot as plt

# Get ground truth labels for the validation set
y_true = training_data.select(pls.col("label"))[validation_split:].to_numpy().flatten()

plt.figure(figsize=(15, 5))
plt.plot(y_true, label='Ground Truth', alpha=0.7)
plt.plot(y_pred, label='Prediction', alpha=0.7)
plt.title('Validation Set: Prediction vs Ground Truth')
plt.xlabel('Sample Index')
plt.ylabel('Label')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import pearsonr

# Calculate Pearson correlation coefficient
pcc, _ = pearsonr(y_true, y_pred)
print("Pearson correlation coefficient (PCC):", pcc)

#### Predictions with test data for submission

In [ ]:
testing_data = pls.read_parquet(Path(data_path, "test.parquet")).drop("label")
testing_data.head(10)

In [ ]:
# predict on the test set
predictions = model.predict(testing_data.select(pls.col(train_feature_list)))
# prepare submission DataFrame
submission_df = pls.DataFrame({
    "ID": np.arange(1, len(predictions)+1),
    "prediction": predictions
})
submission_df.write_csv("submission.csv")
submission_df

In [ ]:
sample_submission = pls.read_csv(Path(data_path, "sample_submission.csv"))
sample_submission.head(10)